### NICF- Intelligent Process Automation (SF)

https://www.iss.nus.edu.sg/executive-education/course/detail/intelligent-process-automation/artificial-intelligence

# Workshop: Text Summarization

### <span style="color:red">Re-created with Python 3 version</span>

In [5]:
! pip install goose3

  Using cached https://files.pythonhosted.org/packages/3b/d4/3b5c17f00cce85b9a1e6f91096e1cc8e8ede2e1be8e96b87ce1ed09e92c5/cssselect-1.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c6/cb/18eeb235f833b726522d7ebed54f2278ce28ba9438e3135ab0278d9792a2/jieba-0.42.1.tar.gz
  Created wheel for jieba: filename=jieba-0.42.1-cp37-none-any.whl size=19314482 sha256=152d48dc7708bb1b8b08ff0a0f290758ea01f7e903c264fadde834aa37872d63
  Stored in directory: C:\Users\guofe\AppData\Local\pip\Cache\wheels\af\e4\8e\5fdd61a6b45032936b8f9ae2044ab33e61577950ce8e0dec29
  Created wheel for nltk: filename=nltk-3.4.5-cp37-none-any.whl size=1449913 sha256=c1ce55828c2ddb5f33eee1d9cc1298c01d98ec962e465904febcc7a752daef12
  Stored in directory: C:\Users\guofe\AppData\Local\pip\Cache\wheels\96\86\f6\68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built jieba nltk


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [6]:
from goose3 import Goose
from collections import Counter
from math import fabs
import re

stopWords = {
    "-", " ", ",", ".", "a", "e", "i", "o", "u", "t", "about", "above",
    "above", "across", "after", "afterwards", "again", "against", "all",
    "almost", "alone", "along", "already", "also", "although", "always",
    "am", "among", "amongst", "amoungst", "amount", "an", "and",
    "another", "any", "anyhow", "anyone", "anything", "anyway",
    "anywhere", "are", "around", "as", "at", "back", "be", "became",
    "because", "become", "becomes", "becoming", "been", "before",
    "beforehand", "behind", "being", "below", "beside", "besides",
    "between", "beyond", "both", "bottom", "but", "by", "call", "can",
    "cannot", "can't", "co", "con", "could", "couldn't", "de",
    "describe", "detail", "did", "do", "done", "down", "due", "during",
    "each", "eg", "eight", "either", "eleven", "else", "elsewhere",
    "empty", "enough", "etc", "even", "ever", "every", "everyone",
    "everything", "everywhere", "except", "few", "fifteen", "fifty",
    "fill", "find", "fire", "first", "five", "for", "former",
    "formerly", "forty", "found", "four", "from", "front", "full",
    "further", "get", "give", "go", "got", "had", "has", "hasnt",
    "have", "he", "hence", "her", "here", "hereafter", "hereby",
    "herein", "hereupon", "hers", "herself", "him", "himself", "his",
    "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed",
    "into", "is", "it", "its", "it's", "itself", "just", "keep", "last",
    "latter", "latterly", "least", "less", "like", "ltd", "made", "make",
    "many", "may", "me", "meanwhile", "might", "mill", "mine", "more",
    "moreover", "most", "mostly", "move", "much", "must", "my", "myself",
    "name", "namely", "neither", "never", "nevertheless", "new", "next",
    "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing",
    "now", "nowhere", "of", "off", "often", "on", "once", "one", "only",
    "onto", "or", "other", "others", "otherwise", "our", "ours",
    "ourselves", "out", "over", "own", "part", "people", "per",
    "perhaps", "please", "put", "rather", "re", "said", "same", "see",
    "seem", "seemed", "seeming", "seems", "several", "she", "should",
    "show", "side", "since", "sincere", "six", "sixty", "so", "some",
    "somehow", "someone", "something", "sometime", "sometimes",
    "somewhere", "still", "such", "take", "ten", "than", "that", "the",
    "their", "them", "themselves", "then", "thence", "there",
    "thereafter", "thereby", "therefore", "therein", "thereupon",
    "these", "they", "thickv", "thin", "third", "this", "those",
    "though", "three", "through", "throughout", "thru", "thus", "to",
    "together", "too", "top", "toward", "towards", "twelve", "twenty",
    "two", "un", "under", "until", "up", "upon", "us", "use", "very",
    "via", "want", "was", "we", "well", "were", "what", "whatever",
    "when", "whence", "whenever", "where", "whereafter", "whereas",
    "whereby", "wherein", "whereupon", "wherever", "whether", "which",
    "while", "whither", "who", "whoever", "whole", "whom", "whose",
    "why", "will", "with", "within", "without", "would", "yet", "you",
    "your", "yours", "yourself", "yourselves", "the", "reuters", "news",
    "monday", "tuesday", "wednesday", "thursday", "friday", "saturday",
    "sunday", "mon", "tue", "wed", "thu", "fri", "sat", "sun",
    "rappler", "rapplercom", "inquirer", "yahoo", "home", "sports",
    "1", "10", "2012", "sa", "says", "tweet", "pm", "home", "homepage",
    "sports", "section", "newsinfo", "stories", "story", "photo",
    "2013", "na", "ng", "ang", "year", "years", "percent", "ko", "ako",
    "yung", "yun", "2", "3", "4", "5", "6", "7", "8", "9", "0", "time",
    "january", "february", "march", "april", "may", "june", "july",
    "august", "september", "october", "november", "december",
    "government", "police"
}
ideal = 20.0


def SummarizeUrl(url):
    try:
        article = grab_link(url)
    except IOError:
        print('IOError')
        return None

    if not (article and article.cleaned_text and article.title):
        return None

    return Summarize(article.title, article.cleaned_text)


def Summarize(title, text):
    sentences = split_sentences(text)
    keys = keywords(text)
    titleWords = split_words(title)

    if len(sentences) <= 5:
        return sentences

    #score setences, and use the top 5 sentences
    sentence_ranks = score(sentences, titleWords, keys)
    return [sentence for sentence, score in sentence_ranks.most_common(5)]

goose = Goose()
def grab_link(url):
    #extract article information using Python Goose
    try:
        article = goose.extract(url=url)
        return article
    except ValueError:
        print('Goose failed to extract article from url')
        return None
    return None


def score(sentences, titleWords, keywords):
    #score sentences based on different features

    senSize = len(sentences)
    ranks = Counter()
    for i, s in enumerate(sentences):
        sentence = split_words(s)
        titleFeature = title_score(titleWords, sentence)
        sentenceLength = length_score(sentence)
        sentencePosition = sentence_position(i+1, senSize)
        sbsFeature = sbs(sentence, keywords)
        dbsFeature = dbs(sentence, keywords)
        frequency = (sbsFeature + dbsFeature) / 2.0 * 10.0

        #weighted average of scores from four categories
        totalScore = (titleFeature*1.5 + frequency*2.0 +
                      sentenceLength*1.0 + sentencePosition*1.0) / 4.0
        ranks[s] = totalScore
    return ranks


def sbs(words, keywords):
    score = 0.0
    if len(words) == 0:
        return 0
    for word in words:
        if word in keywords:
            score += keywords[word]
    return (1.0 / fabs(len(words)) * score)/10.0


def dbs(words, keywords):
    if (len(words) == 0):
        return 0

    summ = 0
    first = []
    second = []

    for i, word in enumerate(words):
        if word in keywords:
            score = keywords[word]
            if first == []:
                first = [i, score]
            else:
                second = first
                first = [i, score]
                dif = first[0] - second[0]
                summ += (first[1]*second[1]) / (dif ** 2)

    # number of intersections
    k = len(set(keywords.keys()).intersection(set(words))) + 1
    return (1/(k*(k+1.0))*summ)


def split_words(text):
    #split a string into array of words
    try:
        text = re.sub(r'[^\w ]', '', text)  # strip special chars
        return [x.strip('.').lower() for x in text.split()]
    except TypeError:
        print("Error while splitting characters")
        return None


def keywords(text):
    """get the top 10 keywords and their frequency scores
    ignores blacklisted words in stopWords,
    counts the number of occurrences of each word
    """
    text = split_words(text)
    numWords = len(text)  # of words before removing blacklist words
    freq = Counter(x for x in text if x not in stopWords)

    minSize = min(10, len(freq))  # get first 10
    keywords = {x: y for x, y in freq.most_common(minSize)}  # recreate a dict

    for k in keywords:
        articleScore = keywords[k]*1.0 / numWords
        keywords[k] = articleScore * 1.5 + 1

    return keywords


def split_sentences(text):
    '''
    The regular expression matches all sentence ending punctuation and splits the string at those points.
    At this point in the code, the list looks like this ["Hello, world", "!" ... ]. The punctuation and all quotation marks
    are separated from the actual text. The first s_iter line turns each group of two items in the list into a tuple,
    excluding the last item in the list (the last item in the list does not need to have this performed on it). Then,
    the second s_iter line combines each tuple in the list into a single item and removes any whitespace at the beginning
    of the line. Now, the s_iter list is formatted correctly but it is missing the last item of the sentences list. The
    second to last line adds this item to the s_iter list and the last line returns the full list.
    '''
    
    sentences = re.split('(?<![A-ZА-ЯЁ])([.!?]"?)(?=\s+\"?[A-ZА-ЯЁ])', text)
    s_iter = list(zip(*[iter(sentences[:-1])] * 2))
    s_iter = [''.join(map(str,y)).lstrip() for y in s_iter]
    s_iter.append(sentences[-1])
    return s_iter



def length_score(sentence):
    return 1 - fabs(ideal - len(sentence)) / ideal


def title_score(title, sentence):
    title = [x for x in title if x not in stopWords]
    count = 0.0
    for word in sentence:
        if (word not in stopWords and word in title):
            count += 1.0
            
    if len(title) == 0:
        return 0.0
        
    return count/len(title)


def sentence_position(i, size):
    """different sentence positions indicate different
    probability of being an important sentence"""

    normalized = i*1.0 / size
    if 0 < normalized <= 0.1:
        return 0.17
    elif 0.1 < normalized <= 0.2:
        return 0.23
    elif 0.2 < normalized <= 0.3:
        return 0.14
    elif 0.3 < normalized <= 0.4:
        return 0.08
    elif 0.4 < normalized <= 0.5:
        return 0.05
    elif 0.5 < normalized <= 0.6:
        return 0.04
    elif 0.6 < normalized <= 0.7:
        return 0.06
    elif 0.7 < normalized <= 0.8:
        return 0.04
    elif 0.8 < normalized <= 0.9:
        return 0.04
    elif 0.9 < normalized <= 1.0:
        return 0.15
    else:
        return 0

### Summerize single web article

In [8]:
# NUS-ISS
# The Straits Times: 3 new programmes to help firms boost digitalisation efforts
url = 'https://www.straitstimes.com/tech/3-new-programmes-to-help-firms-boost-digitalisation-efforts'
summaries = SummarizeUrl(url)
for sentence in summaries:
    print(sentence)
    print('')

Companies looking to boost their digitalisation efforts will now be able to benefit from three new programmes that collectively target workers across all levels of a business.

The courses aim to equip them with the mindset and know-how needed to map out an effective digitalisation strategy for their firms.

The programmes are expected to kick off early next year and are open to SBF members.

Depending on each company's needs, NUS-ISS will then work with them to roll out specific training programmes for their employees.

The programmes will be offered by the Singapore Business Federation (SBF) and the Institute of Systems Science at National University of Singapore (NUS-ISS), with the two organisations signing a memorandum of understanding yesterday.



### Summerize mutiple web articles

In [9]:
from pprint import pprint
urls = (u'https://www.straitstimes.com/tech/3-new-programmes-to-help-firms-boost-digitalisation-efforts',
        u'http://www.huffingtonpost.com/2013/11/22/twitter-forward-secrecy_n_4326599.html',
        u'http://www.bbc.co.uk/news/world-europe-30035666',
        u'http://www.bbc.co.uk/news/magazine-29631332'        
       )

for url in urls:
    print('url : ', url)
    summaries = SummarizeUrl(url)
    pprint(summaries[0:2])
    print('')


url :  https://www.straitstimes.com/tech/3-new-programmes-to-help-firms-boost-digitalisation-efforts
['Companies looking to boost their digitalisation efforts will now be able to '
 'benefit from three new programmes that collectively target workers across '
 'all levels of a business.',
 'The courses aim to equip them with the mindset and know-how needed to map '
 'out an effective digitalisation strategy for their firms.']

url :  http://www.huffingtonpost.com/2013/11/22/twitter-forward-secrecy_n_4326599.html
['"A year and a half ago, Twitter was first served completely over HTTPS," the '
 'company said in a blog posting.',
 '(Reuters) - Twitter Inc said it has implemented a security technology that '
 'makes it harder to spy on its users and called on other Internet firms to do '
 'the same, as Web providers look to thwart spying by government intelligence '
 'agencies.']

url :  http://www.bbc.co.uk/news/world-europe-30035666
['Police, fire officials and a specially-trained hunting

In [10]:
summaries[0]

'The collected ocean plastic would be recycled and made into products - or oil.'

### Summerize local textual article

#### Example from https://www.straitstimes.com/tech/3-new-programmes-to-help-firms-boost-digitalisation-efforts

In [11]:
article_title = u'3 new programmes to help firms boost digitalisation efforts'


In [12]:
article_text = u'Companies looking to boost their digitalisation efforts will now be able to benefit from three new programmes that collectively target workers across all levels of a business. The programmes will be offered by the Singapore Business Federation (SBF) and the Institute of Systems Science at National University of Singapore (NUS-ISS), with the two organisations signing a memorandum of understanding yesterday. Business owners and leaders can sign up for the Digital Competency Roadmap comprising three courses over five days. The courses aim to equip them with the mindset and know-how needed to map out an effective digitalisation strategy for their firms. There is also a longer five-course, 13-day variant for employees at the executive level. Executives who wish to take their classroom learning further can also work towards a Professional Diploma in Digitalisation jointly awarded by SBF and NUS-ISS. This programme will include a four-month project, in which experts from NUS-ISS will guide participants to apply what they have learnt when they are back in their own companies. The last programme, called Skills for Transformation, is targeted at companies that have already started on their digital transformations. Depending on each company\'s needs, NUS-ISS will then work with them to roll out specific training programmes for their employees. The programmes were announced by Minister of State for National Development and Manpower Zaqy Mohamad at the Suntec Singapore Convention and Exhibition Centre during the opening ceremony of SBF\'s Future Economy Conference and Exhibition. "Transformation is an ongoing journey. For companies that have started, you must press on and continue to look out for new opportunities... For those that have yet to start, I encourage you to do so now," he said. The programmes are expected to kick off early next year and are open to SBF members. Those interested in finding out more can attend a 9.30am briefing at the SBF Centre on Nov 20 or register their interest at https://nus.edu/2oNvhfI The programmes are targeted mainly at companies from the retail, logistics, food and beverage, manufacturing and built environment sectors, but SBF is open to considering companies from other industries, said its assistant chief executive Joanne Guo. She added that SkillsFuture subsidies will offset programme costs by up to 90 per cent for small and medium-sized enterprises (SMEs) and 70 per cent for non-SMEs. Mr Zaqy also also provided an update on the Infocomm Media Development Authority\'s Digital Acceleration Index (DAI), revealing that more than 2,000 companies have used the index since its launch earlier this year. The DAI is a self-diagnostic tool that gauges how far along companies are in their digital transformation and provides insights into their digital strengths and weaknesses. The next window for registration will open in January.'


In [13]:
print('<<', article_title, '>>\n')
summaries = Summarize(article_title, article_text)
for sentence in summaries:
    print(sentence)
    print('')

<< 3 new programmes to help firms boost digitalisation efforts >>

Companies looking to boost their digitalisation efforts will now be able to benefit from three new programmes that collectively target workers across all levels of a business.

The courses aim to equip them with the mindset and know-how needed to map out an effective digitalisation strategy for their firms.

The programmes will be offered by the Singapore Business Federation (SBF) and the Institute of Systems Science at National University of Singapore (NUS-ISS), with the two organisations signing a memorandum of understanding yesterday.

The DAI is a self-diagnostic tool that gauges how far along companies are in their digital transformation and provides insights into their digital strengths and weaknesses.

Depending on each company's needs, NUS-ISS will then work with them to roll out specific training programmes for their employees.



In [14]:
print('[ most important sentence ]\n', summaries[0])

[ most important sentence ]
 Companies looking to boost their digitalisation efforts will now be able to benefit from three new programmes that collectively target workers across all levels of a business.


#### Example from https://www.iss.nus.edu.sg/executive-education/course/detail/intelligent-process-automation/artificial-intelligence

In [15]:
article_title = u'NICF- Intelligent Process Automation (SF)'
article_text = u'Overview. This 3-day course enables participants to create intelligent virtual workers (intelligent agent) able to mimic real person to take business related actions and procedures by operating a computer and computer applications, in various business processes and industry domains. Participants learn practical knowledge of intelligent agent (IA), artificial intelligence (AI), open source toolsets, and computer scripting skills to design and create intelligent virtual workers as a minimal viable product (MVP) to automate business processes. Application examples are: Virtual employee for account management, e.g. mortgage loan application, invoice processing; Interactive messaging/chat-bot for customer service; request fulfillment; frequently asked questions; Intelligent software agents in research, innovation and video game industry; This course can benefit job roles such as Automation Managers, Enterprise Architects, Application Solution Architects, System Integrator, Technology Consultants, Senior Process Analysts, Senior Process Engineers, and working professionals who intend to design, develop, implement and evaluate various applications of cognitive robotic process automation with artificial intelligence. Participants will benefit from a careful balance of lectures and practical workshops. Some of the topics covered include concepts and techniques of robotic process automation; artificial intelligence fundamentals; enterprise system integration; email automation; AI processing of image, speech, language, sentiment analysis; and interactive conversational virtual worker/assistant. There will be projects and assessment to reinforce participants’ learning as part of the course. This course is a part of Graduate Certificate in Intelligent Software Agents, which is a part of the Stackable Graduate Certificate Programme in Artificial Intelligent Systems offered by NUS-ISS. Key Takeaways. Upon completion of the course, students will be able to: Identify enterprise business use cases and applications to implement intelligent process automation (IPA), leveraging technologies of intelligent software agent and process automation. Evaluate intelligent process automation techniques, including desktop automation, web automation, visual user interface automation, spoken language and written document understanding, sentiment analysis, image text extraction, customer face detection, customer speech recognition, task agent interaction and automation through voice commands. Architect IPA applications useful to learner’s organization using tools: E.g. Virtual workers/assistants for customer care. E.g. Enterprise visual and user interface automation applications using artificial intelligence, chat-bot, and computer scripting for mortgage application automation. E.g. Reinforcement learning agent for research, innovation and video game industry. Develop IPA Minimum Viable Product (MVP). Who Should Attend. This course is suitable for both information technology and non-IT professionals who are interested in creating intelligent robots, or virtual workers, able to mimic real personnel to automatically take business related actions and procedures by operating a computer, in various business processes and industry domains. This course will be useful for: Automation Manager who want to improve personnel productivity. Enterprise Architects & Application Solution Architects who want to integrate intelligent process automation into their solution. System Integrators & Technology Consultants who want to use this technology for system integration. Senior Process Analysts & Senior Process Engineers who want to customize, configure and use virtual worker. Robotic Process Automation practitioners who want to leverage artificial intelligence technology. Prerequisites. This is a intermediate course involves rapid prototyping using several computer programing, e.g. graphical programming, JavaScript, python, APIs, and cloud computing. Participants with prior computer programming experiences would benefit more during hands-on workshops and project submission as part of course assessment. Participants should preinstall and successfully run iss-vm virtual machine (about 30 GB in size) into their own device before course start date. Participants should have basic computer literacy and software engineering fundamentals, e.g. using Windows or Linux or MacOS, Internet, Email, Spreadsheet, VMware or VirtualBox, and aware of cloud computing, application programming interface (API), and client-server software architecture. Participants should have basic hands-on coding experience in one or more high-level computer programming languages, preferable in JavaScript and Python. Basic knowledge of pattern recognition at the level of NICF- Problem Solving using Pattern Recognition (SF). Note: Participants are required to bring their own internet enabled computing device (laptops, tablet etc) & power charger to access and download electronic courseware in PDF format. We will be only be providing courseware in PDF and will not issue any printed copies.'


In [16]:
print('<<', article_title, '>>\n')
summaries = Summarize(article_title, article_text)
for sentence in summaries:
    print(sentence)
    print('')

<< NICF- Intelligent Process Automation (SF) >>

Evaluate intelligent process automation techniques, including desktop automation, web automation, visual user interface automation, spoken language and written document understanding, sentiment analysis, image text extraction, customer face detection, customer speech recognition, task agent interaction and automation through voice commands.

Upon completion of the course, students will be able to: Identify enterprise business use cases and applications to implement intelligent process automation (IPA), leveraging technologies of intelligent software agent and process automation.

Enterprise Architects & Application Solution Architects who want to integrate intelligent process automation into their solution.

Some of the topics covered include concepts and techniques of robotic process automation; artificial intelligence fundamentals; enterprise system integration; email automation; AI processing of image, speech, language, sentiment anal

In [17]:
print('[ most important sentence ]\n', summaries[0])

[ most important sentence ]
 Evaluate intelligent process automation techniques, including desktop automation, web automation, visual user interface automation, spoken language and written document understanding, sentiment analysis, image text extraction, customer face detection, customer speech recognition, task agent interaction and automation through voice commands.


---

## <span style="color:red">Your Exercise:</span> 

## Apply the text summarization technology to summarize textual data, e.g. emails, customer feedbacks, news articles, competitor web pages.


In [18]:
url = 'https://www.nytimes.com/2020/03/11/business/boeing-737-max-wire-bundles.html'
summaries = SummarizeUrl(url)
for sentence in summaries:
    print(sentence)
    print('')

 The wire bundles were not implicated in those accidents, which have been linked to problems with a software system known as MCAS.

The wire bundles have raised concerns because they could, in rare circumstances, cause a short circuit and possibly lead to a catastrophic failure.

Boeing has argued privately to regulators that the likelihood of such a failure is remote.

Boeing had hoped to avoid having to uncover and separate the wiring, but concluded that global regulators’ insistence on it might further delay the Max’s return, the source said.

The Max was grounded a year ago after it was involved in two crashes that killed a total of 346 people.



---